# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Craig Wilcox

### GitHub Repository: https://github.com/s256657/json-sentiment

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

#### Prework import requirements

In [11]:
# Create and activate a Python virtual environment. 
# Before starting the project, try all these imports FIRST
# Address any errors you get running this code cell 
# by installing the necessary packages into your active Python environment.
# Try to resolve issues using your materials and the web.
# If that doesn't work, ask for help in the discussion forums.
# You can't complete the exercises until you import these - start early! 
# We also import json and pickle (included in the Python Standard Library).

import json
import pickle

import requests
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

print('All prereqs installed.')
!pip list

All prereqs installed.
Package                 Version
----------------------- -----------
annotated-types         0.7.0
appnope                 0.1.4
asttokens               3.0.0
blis                    1.3.0
catalogue               2.0.10
certifi                 2025.7.14
charset-normalizer      3.4.2
click                   8.2.1
cloudpathlib            0.21.1
comm                    0.2.2
confection              0.1.5
cymem                   2.0.11
debugpy                 1.8.14
decorator               5.2.1
en_core_web_sm          3.8.0
executing               2.2.0
idna                    3.10
ipykernel               6.29.5
ipython                 9.4.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
Jinja2                  3.1.6
joblib                  1.5.1
jupyter_client          8.6.3
jupyter_core            5.8.1
langcodes               3.5.0
language_data           1.3.0
marisa-trie             1.2.1
markdown-it-py          3.0.0
MarkupSafe              3.0.2


In [13]:
# Version check to confirm spacy and spacyblob are working properly

import spacy
import spacytextblob

print(spacy.__version__)
print(spacytextblob.__version__)

3.8.7
5.0.0


### 1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [18]:
import requests
import json

result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

### 2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [28]:
import spacy
from textblob import TextBlob
from spacy.tokens import Doc

# Step 1: Load lyrics from the result dictionary
lyrics = result.get("lyrics", "")

# Step 2: Load spaCy model
nlp = spacy.load("en_core_web_sm")
doc = nlp(lyrics)

# Step 3: Manually calculate polarity using TextBlob
blob = TextBlob(lyrics)
polarity = blob.sentiment.polarity

# Step 4: Manually register and set polarity extension
if not Doc.has_extension("polarity"):
    Doc.set_extension("polarity", default=None)

doc._.polarity = polarity

# Step 5: Print the result
print("Polarity score:", doc._.polarity)

# Optional: Print lyrics preview
print("\nLyrics preview:\n", lyrics[:300])

Polarity score: 0.04505208333333333

Lyrics preview:
 I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not actually your friend 
But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee


### 3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [17]:
import requests
import json

def save_lyrics_to_file(artist, song, filename):
    """
    Fetches lyrics from lyrics.ovh API and writes to a JSON file.

    Args:
        artist (str): Artist name
        song (str): Song title
        filename (str): File path to save lyrics JSON
    """
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    response = requests.get(url)

    if response.status_code == 200:
        try:
            data = response.json()
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            print(f"Lyrics saved to {filename}")
        except json.JSONDecodeError:
            print("Error: Response is not valid JSON.")
    else:
        print(f"Failed to fetch lyrics for '{song}' by '{artist}'. Status code: {response.status_code}")

# Test with 4 songs
save_lyrics_to_file("Queen", "Bohemian Rhapsody", "bohemian_rhapsody.json")
save_lyrics_to_file("Adele", "Hello", "hello_adele.json")
save_lyrics_to_file("Billy Joel", "Piano Man", "piano_man.json")
save_lyrics_to_file("Coldplay", "Yellow", "yellow_coldplay.json")

Failed to fetch lyrics for 'Bohemian Rhapsody' by 'Queen'. Status code: 504
Lyrics saved to hello_adele.json
Lyrics saved to piano_man.json
Lyrics saved to yellow_coldplay.json


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.